#### Run-Day Breakout System
+ Run days occur in trending markets
+ Buy reversals are initiated when market closes above the true high of specified number of prior down run days
+ Sell reversals are initiated when market closes below the true low of specified number of prior up days

#### Up Run Day
+ The true high of the run day is greater than the maximum true high of past N days
+ The true low of the run day is less than the minimum true low on the subsequent N days

#### Down Run Day
+ The true low of the run day is less then the minimum of true low of the past N days
+ The true high of the run day is greater than the maximum true high of the subsequent N days

#### Why this could work
+ The ability for market to close opposite extreme point of strongly trending days implies possible trend reversal

#### Trading Signals
+ Buy
 + Close is above maximum true high of most recent N down run days
 + The most recent run day must be an up run day

+ Sell
    + Close is below minimum of true low of most recent up run days 
    + The most recent run day muse me a down run day
    
#### Parameters
+ N1: run day if daily true high/low is above/below following N1 days
+ N2: number of prior run days used to compute the max true high/min true low for buy/sell signal


#### Daily Checklist
+ Check whether the trading day N days prior to current day can defined as 
+ Track all Run Days and their true high and lows
+ If short:
    + Check whether today's close is above the max true high of past N down run days
    + If True and most recent run day was up run day reverse from short to long
+ If long:
    + Check whether today's close is below the min true low of past N up run days
    + If True and most recent run day was down run day, reverse from long to short 

+ If a day meets the criteria of being both an up and down run day, then it is not considered a run day

In [2]:
from math import floor
import datetime as datetime
import quandl as ql
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pylab import mpl, plt
plt.style.use('seaborn')
mpl.rcParams['font.family'] = 'serif'
k = 'YgkL6nxrm_SsUs7so2MM'

In [3]:
cl = ql.get('CHRIS/CME_CL1', api_key=k)

In [4]:
cl.tail()

,Open,High,Low,Last,Change,Settle,Volume,Previous Day Open Interest
Date,,,,,,,,
2019-01-18,52.23,53.92,52.09,53.76,1.73,53.80,186006.0,61293.0
2019-01-22,53.73,54.24,51.80,52.77,1.23,52.57,39659.0,25682.0
2019-01-23,52.95,53.64,51.86,52.40,0.39,52.62,698010.0,499043.0
2019-01-24,52.45,53.47,52.07,53.18,0.51,53.13,708068.0,496877.0
2019-01-25,53.17,53.94,52.91,53.55,0.56,53.69,587559.0,481966.0


In [6]:
#def compute_run_days(df):
df = cl   
true_range = df.copy()
possible_highs = pd.concat([df['Last'].shift(1), df['High']],axis=1)
true_range['true_high'] = np.max(possible_highs,axis=1)

possible_lows = pd.concat([df['Low'],df['Last'].shift(1)],axis=1)
true_range['true_low'] = np.min(possible_lows, axis=1)
true_range['true_range'] = true_range['true_high'] - true_range['true_low']


In [26]:
N = 2
i = 55
#interval_low
# for i in range(len(true_range)):
#     interval_high = true_range[i:i+2]


In [27]:
true_range[i:i+2]

,Open,High,Low,Last,Change,Settle,Volume,Previous Day Open Interest,true_high,true_low,true_range
Date,,,,,,,,,,,
1983-06-17,31.60,31.60,31.40,31.40,NaN,31.40,128.0,250.0,31.60,31.40,0.20
1983-06-20,31.37,31.58,31.37,31.42,NaN,31.42,188.0,1497.0,31.58,31.37,0.21


In [29]:
true_range[i-N:i]

,Open,High,Low,Last,Change,Settle,Volume,Previous Day Open Interest,true_high,true_low,true_range
Date,,,,,,,,,,,
1983-06-15,31.1,31.30,31.1,31.3,NaN,31.3,157.0,264.0,31.30,31.0,0.30
1983-06-16,31.3,31.65,31.3,31.6,NaN,31.6,150.0,231.0,31.65,31.3,0.35


In [34]:
true_range.iloc[33:34].

,Open,High,Low,Last,Change,Settle,Volume,Previous Day Open Interest,true_high,true_low,true_range
Date,,,,,,,,,,,
1983-05-17,30.05,30.2,29.9,29.91,NaN,29.91,124.0,80.0,30.2,29.9,0.3


In [87]:
# true_range_sub = true_range.iloc[30:40]
true_range_sub['run_day'] = 0.0

/Users/justinsimcock/miniconda3/envs/py35_trading/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [92]:
N = 5
for i in range(len(true_range)):
    #up
    true_range_low_up = true_range[i:i+N].true_low.min()
    true_range_high_up = true_range[i-N:i].true_high.max()

    #down
    true_range_high_down = true_range[i:i+N].true_high.max()
    true_range_low_down = true_range[i-N:i].true_low.min()

    #uprun
    if true_range.iloc[i-N].High > true_range.iloc[i-2*N:i-N].High.max() and true_range.iloc[i-N].Low < true_range.iloc[i-N:i].Low.min():
        true_range.loc[true_range.index[i-N],'run_day'] = 1.0

    #downrun
    elif true_range.iloc[i-N].Low < true_range.iloc[i-2*N:i-N].Low.min() and true_range.iloc[i-N].High > true_range.iloc[i-N:i].High.max(): 
        true_range.loc[true_range.index[i-N], 'run_day'] = -1.0
#     else: true_range.loc[true_range.index[i], 'run_day'] = 0.0


In [66]:
true_range.loc[true_range.index[i], 'run_day'] = 0

In [93]:
true_range['run_day'].unique()

array([0., 1.])

In [91]:
true_range.loc['2014-11-25':'2014-12-31']

,Open,High,Low,Last,Change,Settle,Volume,Previous Day Open Interest,true_high,true_low,true_range,run_day
Date,,,,,,,,,,,,
2014-11-25,75.51,76.58,73.71,74.09,1.69,74.09,294447.0,333880.0,76.58,73.71,2.87,0.0
2014-11-26,73.87,74.48,73.30,73.69,0.40,73.69,258349.0,332646.0,74.48,73.30,1.18,0.0
2014-11-28,73.50,73.56,65.69,66.15,7.54,66.15,552823.0,334009.0,73.69,65.69,8.00,0.0
2014-12-01,66.00,69.54,63.72,66.54,2.85,69.00,520303.0,342057.0,69.54,63.72,5.82,0.0
2014-12-02,69.24,69.32,66.67,66.88,2.12,66.88,360290.0,329651.0,69.32,66.54,2.78,0.0
2014-12-03,67.60,68.23,66.80,67.38,0.50,67.38,293484.0,319555.0,68.23,66.80,1.43,0.0
2014-12-04,67.40,68.22,66.09,66.81,0.57,66.81,237011.0,309609.0,68.22,66.09,2.13,0.0
2014-12-05,66.80,66.89,65.17,65.84,0.97,65.84,276529.0,301596.0,66.89,65.17,1.72,0.0
2014-12-08,65.46,65.55,62.78,63.05,2.79,63.05,330077.0,276921.0,65.84,62.78,3.06,0.0
